In [278]:
import math
import struct

import pandas
import matplotlib.pyplot as plt

In [279]:
df = pandas.read_csv("Input/dblp_names.csv", error_bad_lines=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8656\2959035247.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pandas.read_csv("Input/dblp_names.csv", error_bad_lines=False)
Skipping line 164: expected 2 fields, saw 3
Skipping line 281: expected 2 fields, saw 3
Skipping line 512: expected 2 fields, saw 3
Skipping line 839: expected 2 fields, saw 3
Skipping line 854: expected 2 fields, saw 3
Skipping line 1997: expected 2 fields, saw 3
Skipping line 2631: expected 2 fields, saw 3
Skipping line 2642: expected 2 fields, saw 3
Skipping line 2656: expected 2 fields, saw 3
Skipping line 2658: expected 2 fields, saw 3
Skipping line 2659: expected 2 fields, saw 3
Skipping line 6310: expected 2 fields, saw 3
Skipping line 10753: expected 2 fields, saw 3
Skipping line 11044: expected 2 fields, saw 3
Skipping line 18510: expected 2 fields, saw 3
Skipping line 31242: expected 2 fields, saw 3
Skippi

In [280]:
df.describe()

,# OLD_NAME,NEW_NAME
count,45156,45156
unique,44844,36032
top,Y. Choi,Alberto L. Sangiovanni-Vincentelli
freq,4,9


In [281]:
df_test = df

In [282]:
def get_last_name(s: str) -> str:
    bad_chars = [')', '(']
    s = ''.join((filter(lambda i: i not in bad_chars, s)))
    elements = s.strip().split(' ')
    return elements[len(elements)-1]

In [283]:
def get_first_letter(s:str) -> str:
    return s[0]

In [284]:
df.columns

Index(['# OLD_NAME', ' NEW_NAME'], dtype='object')

In [285]:
df_test['s1_old'] = df['# OLD_NAME'].apply(get_last_name)

In [286]:
def strategy2(s: str) -> str:
    return get_first_letter(s) + get_first_letter(get_last_name(s))

In [287]:
df_test['s2_old'] = df_test['# OLD_NAME'].apply(strategy2)

In [288]:
df_test['s1_new'] = df_test[' NEW_NAME'].apply(get_last_name)
df_test['s2_new'] = df_test[' NEW_NAME'].apply(strategy2)

In [289]:
import numpy as np

df_test['matching_s1'] = np.where((df['s1_new'] == df['s1_old']), True, False)
df_test['matching_s2'] = np.where((df['s2_new'] == df['s2_old']), True, False)

In [290]:
df_test.head()

,# OLD_NAME,NEW_NAME,s1_old,s2_old,s1_new,s2_new,matching_s1,matching_s2
0,Bamshad Mobasherm,Bamshad Mobasher,Mobasherm,BM,Mobasher,BM,False,True
1,Carole Delporte-Gellet,Carole Delporte-Gallet,Delporte-Gellet,CD,Delporte-Gallet,CD,False,True
2,Ronald Presott Loui,Ronald Prescott Loui,Loui,RL,Loui,RL,True,True
3,Eve Riskin,Eve A. Riskin,Riskin,ER,Riskin,ER,True,True
4,Doouglas W. Clark,Douglas W. Clark,Clark,DC,Clark,DC,True,True


In [291]:
df_test['matching_s1'].value_counts()/len(df_test['# OLD_NAME'])

True     0.763221
False    0.236779
Name: matching_s1, dtype: float64

In [292]:
df_test['matching_s2'].value_counts()/len(df_test['# OLD_NAME'])

True     0.905372
False    0.094628
Name: matching_s2, dtype: float64

In [293]:
def function_to_calculate_save(df: pandas.DataFrame):
    n_pairs_before = (45156 * (45156 - 1)) / 2
    n_pairs_after = 0
    for first, second in df.iterrows():
        n_pairs_in_block = (second.iloc[-1] * (second.iloc[-1]- 1)) / 2
        n_pairs_after += n_pairs_in_block

    # Calculate Save for the blocking strategy
    save = 1 - (n_pairs_after / n_pairs_before)
    return save

In [294]:
df_combined = pandas.concat([df_test['s1_old'], df_test['s1_new']])
df_s1 = pandas.DataFrame(df_combined.value_counts())

In [295]:
save = function_to_calculate_save(df_s1)

In [296]:
save

0.9984685960629365

In [297]:
df_combined = pandas.concat([df_test['s2_old'], df_test['s2_new']])
df_s2 = pandas.DataFrame(df_combined.value_counts())
save = function_to_calculate_save(df_s2)

In [298]:
save

0.9867491624085655

The results differ because s2 has a broader criteria for adding a name to a block compared to the s1. It leads to a higher recall, since there is a higher possibility for a name to be put into the same block.
The first one has a stricter selection criteria and the probability to end up in a particular block is lower. It leads to a smaller recall.

Save is high in both cases. Yet, the save of the s2 is slightly smaller. We think it's because larger blocks lead to smaller savings in the number of similarity computations. This might be because there are more pairs of mentions that need to be compared for similarity.

We would prefer the strategy s1.

Two possible properties that can affect the strategies are the distribution of the names and the variation of names in terms of spelling or formatting. A low variation in names would lead to fewer blocks being created, as more names get put into the same block leading to lower saves. For the variation of names it would be bad if the names between old and new names are formatted or spelled very differently, as this might lead to names being put into different blocks which results in a lower recall.